In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from astropy.io import fits

from pipeline.database import DatabaseInterface, register_fits_files
from pipeline.reducer import reduce_science_file

In [2]:
workdir = Path('workdir')
db_path = workdir / 'db.sqlite3'
raw_path = workdir / 'raw'

if db_path.exists(): db_path.unlink()

# 1 register our raw frames into the database

In [3]:
register_fits_files(raw_path, db_path)

# 2 get the science files that have not been reduced yet

In [4]:
database = DatabaseInterface(db_path)

In [5]:
database.get_unreduced_science_files()

,unique_key
0,PG1047__2022-03-10T04:05:51.491
1,PG1047__2022-03-10T04:06:39.568
2,PG1047__2022-03-10T04:07:26.965
3,PG1047__2022-03-10T04:08:13.475
4,SN2018ibb__2022-03-10T00:37:36.811
5,SN2018ibb__2022-03-10T00:44:57.559
6,SN2018ibb__2022-03-10T00:52:18.441
7,SN2018ibb__2022-03-10T00:59:38.971
8,SN2021qvv__2022-03-10T04:23:16.247
9,SN2021qvv__2022-03-10T04:26:20.038


# 3 reduce one science file

In [6]:
science_id = 'PG1047__2022-03-10T04:05:51.491'
science_file, flats, darks = database.get_files_for_science(science_id)

In [7]:
# let us see what flats we have available.
flats

,id,path,binning,filter,category,type,mjd,exposure_time,read_speed,object_name,date
0,1,workdir/raw/EFOSC.2022-03-09T23:21:40.803.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.973389,1.0003,fastL,"SKY,FLAT",2022-03-09T23:21:44.772
1,4,workdir/raw/EFOSC.2022-03-09T23:23:33.956.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.974699,4.5019,fastL,"SKY,FLAT",2022-03-09T23:23:41.185
2,10,workdir/raw/EFOSC.2022-03-09T23:24:12.950.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.975150,5.1009,fastL,"SKY,FLAT",2022-03-09T23:24:21.037
3,17,workdir/raw/EFOSC.2022-03-09T23:24:52.956.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.975613,5.9018,fastL,"SKY,FLAT",2022-03-09T23:25:01.794
4,18,workdir/raw/EFOSC.2022-03-09T23:22:55.941.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.974259,3.9010,fastL,"SKY,FLAT",2022-03-09T23:23:02.450
5,20,workdir/raw/EFOSC.2022-03-09T23:22:17.907.fits,2x2,U#640,CALIB,"SKY,FLAT",59647.973818,3.4017,fastL,"SKY,FLAT",2022-03-09T23:22:23.891


In [8]:
output_dir = workdir / 'reduced'
output_dir.mkdir(exist_ok=True)

output_file = output_dir / f"{science_id}.fits"
reduce_science_file(science_file, flats['path'], darks['path'], output_file)

In [9]:
#reduced = fits.getdata(output_file)[5:900, 5:900]
#from astropy.visualization import simple_norm
#norm = simple_norm(reduced, percent=98)
#plt.imshow(reduced, norm=norm, origin='lower')

Tell the database we reduced this science file.

In [10]:
database.add_reduced_data(science_id, output_file);

Now the file does not come up anymore when querying unreduced science files.

In [11]:
database.get_unreduced_science_files()

,unique_key
0,PG1047__2022-03-10T04:06:39.568
1,PG1047__2022-03-10T04:07:26.965
2,PG1047__2022-03-10T04:08:13.475
3,SN2018ibb__2022-03-10T00:37:36.811
4,SN2018ibb__2022-03-10T00:44:57.559
5,SN2018ibb__2022-03-10T00:52:18.441
6,SN2018ibb__2022-03-10T00:59:38.971
7,SN2021qvv__2022-03-10T04:23:16.247
8,SN2021qvv__2022-03-10T04:26:20.038
9,SN2021qvv__2022-03-10T04:29:24.507


# 4 Reduce all science files in the same way

In [12]:
for science_id in database.get_unreduced_science_files()['unique_key']:
    print('reducing', science_id)
    science_file, flats, darks = database.get_files_for_science(science_id)
    if len(flats) == 0 or len(darks) == 0:
        print('   not enough calibs!')
        continue
    output_file = output_dir / f"{science_id}.fits"
    reduce_science_file(science_file, flats['path'], darks['path'], output_file)
    database.add_reduced_data(science_id, output_file)

reducing PG1047__2022-03-10T04:06:39.568
reducing PG1047__2022-03-10T04:07:26.965
   not enough calibs!
reducing PG1047__2022-03-10T04:08:13.475
   not enough calibs!
reducing SN2018ibb__2022-03-10T00:37:36.811
reducing SN2018ibb__2022-03-10T00:44:57.559
reducing SN2018ibb__2022-03-10T00:52:18.441
reducing SN2018ibb__2022-03-10T00:59:38.971
reducing SN2021qvv__2022-03-10T04:23:16.247
reducing SN2021qvv__2022-03-10T04:26:20.038
reducing SN2021qvv__2022-03-10T04:29:24.507
reducing SN2021qvv__2022-03-10T04:32:28.168






Executing the same sequence again, nothing is done, saving CPU time.

In [13]:
for science_id in database.get_unreduced_science_files()['unique_key']:
    print('reducing', science_id)
    science_file, flats, darks = database.get_files_for_science(science_id)
    if len(flats) == 0 or len(darks) == 0:
        print('   not enough calibs!')
        continue
    output_file = output_dir / f"{science_id}.fits"
    reduce_science_file(science_file, flats['path'], darks['path'], output_file)
    database.add_reduced_data(science_id, output_file)

reducing PG1047__2022-03-10T04:07:26.965
   not enough calibs!
reducing PG1047__2022-03-10T04:08:13.475
   not enough calibs!
